In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

In [2]:
# 处理原始数据，然后合并
# 读取
df_wf = pd.read_table('../Data/House_ShangYu/wufang.txt', low_memory=False, dtype=float)
df_yf = pd.read_csv('../Data/House_ShangYu/youfang.csv', low_memory=False, dtype=float)
# 填充原始nan
df_wf = df_wf.fillna(0)
df_yf = df_yf.fillna(0)
# 处理顺序
df_wf = df_wf.reindex(columns=list(["ID","SB","GJJ","NL","CL","HY","GS","SS","Y"]))
df_yf = df_yf.reindex(columns=list(["ID","SB","GJJ","NL","CL","HY","GS","SS","Y"]))
# 填充 Y 列的目标值
df_wf["Y"] = df_wf["Y"].fillna(0)
df_yf["Y"] = df_yf["Y"].fillna(1)
# 合并，有房+0.3无房
df_wf_train, df_wf_test = train_test_split(
    df_wf, test_size=0.7)

df_house = pd.concat([df_wf_train,df_yf])
df_house = df_house.sample(frac=1)

In [3]:
df_house.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17665 entries, 4983 to 7749
Data columns (total 9 columns):
ID     17665 non-null float64
SB     17665 non-null float64
GJJ    17665 non-null float64
NL     17665 non-null float64
CL     17665 non-null float64
HY     17665 non-null float64
GS     17665 non-null float64
SS     17665 non-null float64
Y      17665 non-null float64
dtypes: float64(9)
memory usage: 1.3 MB


In [4]:
df_house_x = df_house.drop(["Y", "ID"],axis=1)
df_house_view  = df_house_x;
df_house_y = df_house['Y']
df_wf_x = df_wf.drop(["Y", "ID"],axis=1)
df_wf_view  = df_wf_x;
df_wf_y = df_wf['Y']

In [10]:
# 独热编码
# np_house_y_oh = (np.arange(2) == df_house_y[:,None]).astype(np.float32)
# verify_Y_oh = (np.arange(2) == verify_Y[:,None]).astype(np.float32)
# print(type(np_house_y_oh))

<class 'numpy.ndarray'>


In [13]:
# df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(
#     df_house_x, df_house_y, test_size=0.7)
# np_yf_x_train, np_yf_x_test, np_yf_y_train, np_yf_y_test = train_test_split(
#     df_yf_x, df_yf_y, test_size=0.7)
# np_x_train = np.append(np_wf_x_train, np_yf_x_train, axis=0)
# np_x_test = np.append(np_wf_x_test, np_yf_x_test, axis=0)
# np_y_train = np_y_train.values.reshape(-1,1)
# np_y_test = np_y_test.values.reshape(-1,1)

In [12]:
print(type(np_x_train))

<class 'pandas.core.frame.DataFrame'>


In [6]:
# 独热编码
df_house_y_oh = (np.arange(2) == df_house_y[:,None]).astype(np.float32)
df_wf_y_oh = (np.arange(2) == df_wf_y[:,None]).astype(np.float32)

In [15]:
print(np_y_test.shape)
print(np_y_train.shape)
print(np_x_train.shape)
print(np_x_test.shape)


(24682,)
(10578,)
(10578, 7)
(24682, 7)


In [5]:
ss = StandardScaler()
mm = MinMaxScaler()
np_house_x = mm.fit_transform(df_house_x)
df_house_x = pd.DataFrame(np_house_x)
np_wf_x = mm.fit_transform(df_wf_x)
df_wf_x = pd.DataFrame(np_wf_x)


In [7]:
input_size = 7
output_size = 2
batch_size = 100
# placeholder
x = tf.placeholder(tf.float32, [None, input_size], name='x')
y = tf.placeholder(tf.float32, [None, output_size], name='y')

# Variable
W1 = tf.Variable(
    tf.random_normal([input_size, 10], stddev=0.03), name='W1')
b1 = tf.Variable(tf.zeros([10]), name='b1')
h1 = tf.nn.relu(tf.add(tf.matmul(x, W1), b1))

W2 = tf.Variable(
    tf.random_normal([10, 5], stddev=0.03), name='W2')
b2 = tf.Variable(tf.zeros([5]), name='b2')
h2= tf.nn.relu(tf.add(tf.matmul(h1, W2), b2))


W3 = tf.Variable(
    tf.random_normal([5, output_size], stddev=0.03), name='W3')
b3 = tf.Variable(tf.zeros([output_size]), name='b3')


ho = tf.add(tf.matmul(h2, W3), b3)

prediction = tf.nn.softmax(ho)
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))

optimizer = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
acc_op = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [14]:
# tf.reset_default_graph()
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    
    total_batch_len = int(len(df_house_x) / batch_size)
    for i in range(3000):
#         for j  in range(total_batch_len):
#             batch_x = df_house_x[j*batch_size : (j+1)*batch_size]
#             batch_y = df_house_y_oh[j*batch_size : (j+1)*batch_size]
#             _,loss = sess.run([optimizer,cross_entropy],feed_dict={x:batch_x,y:batch_y})
        _,loss = sess.run([optimizer,cross_entropy],feed_dict={x:df_house_x,y:df_house_y_oh})
        if(i%50==0):
            print("i:", i, "loss = ", "{:.3f}".format(loss))
    
#     acc = sess.run(accuracy, feed_dict={x: np_x_test, y: np_y_test})
#     print(acc)
    pre = sess.run(prediction,feed_dict={x:df_wf_x})
#     print(pre)
    
    accuracy = sess.run(acc_op,feed_dict={x:df_wf_x,y:df_wf_y_oh})
    print("Accuracy on validation set: %.9f" % accuracy)

i: 0 loss =  0.693
i: 50 loss =  0.670
i: 100 loss =  0.603
i: 150 loss =  0.553
i: 200 loss =  0.543
i: 250 loss =  0.536
i: 300 loss =  0.525
i: 350 loss =  0.510
i: 400 loss =  0.497
i: 450 loss =  0.487
i: 500 loss =  0.480
i: 550 loss =  0.474
i: 600 loss =  0.468
i: 650 loss =  0.461
i: 700 loss =  0.454
i: 750 loss =  0.447
i: 800 loss =  0.441
i: 850 loss =  0.437
i: 900 loss =  0.434
i: 950 loss =  0.433
i: 1000 loss =  0.432
i: 1050 loss =  0.431
i: 1100 loss =  0.430
i: 1150 loss =  0.430
i: 1200 loss =  0.429
i: 1250 loss =  0.429
i: 1300 loss =  0.429
i: 1350 loss =  0.428
i: 1400 loss =  0.428
i: 1450 loss =  0.428
i: 1500 loss =  0.428
i: 1550 loss =  0.428
i: 1600 loss =  0.428
i: 1650 loss =  0.428
i: 1700 loss =  0.427
i: 1750 loss =  0.427
i: 1800 loss =  0.427
i: 1850 loss =  0.427
i: 1900 loss =  0.427
i: 1950 loss =  0.427
i: 2000 loss =  0.427
i: 2050 loss =  0.427
i: 2100 loss =  0.427
i: 2150 loss =  0.427
i: 2200 loss =  0.427
i: 2250 loss =  0.427
i: 2300 los

In [ ]:
# print(pre.shape)
# print(pre)
# accuracy_score(np_y_test, pre)
from sklearn import metrics
from matplotlib import rcParams
# 1、调用函数计算micro类型的AUC
print('调用函数auc：', metrics.roc_auc_score(df_wf_y_oh, pre, average='micro'))
# 2、手动计算micro类型的AUC
#首先将矩阵y_one_hot和y_score展开，然后计算假正例率FPR和真正例率TPR
fpr, tpr, thresholds = metrics.roc_curve(df_wf_y_oh.ravel(),pre.ravel())
auc = metrics.auc(fpr, tpr)
print('手动计算auc：', auc)
#绘图
rcParams['font.sans-serif'] = u'SimHei'
rcParams['axes.unicode_minus'] = False
#FPR就是横坐标,TPR就是纵坐标
plt.plot(fpr, tpr, c = 'r', lw = 2, alpha = 0.7, label = u'AUC=%.3f' % auc)
plt.plot((0, 1), (0, 1), c = '#808080', lw = 1, ls = '--', alpha = 0.7)
plt.xlim((-0.01, 1.02))
plt.ylim((-0.01, 1.02))
plt.xticks(np.arange(0, 1.1, 0.1))
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('False Positive Rate', fontsize=13)
plt.ylabel('True Positive Rate', fontsize=13)
plt.grid(b=True, ls=':')
plt.legend(loc='lower right', fancybox=True, framealpha=0.8, fontsize=12)
plt.title(u'XGB后的ROC和AUC', fontsize=17)
plt.show()

In [36]:
correct_pred =  np.argmax(pre, 1)
print(correct_pred.shape)
print(np_y_test.shape)

(19450,)
(19450,)


In [37]:
accuracy_score(correct_pred, np_y_test)

0.6996915167095116

In [ ]:
from sklearn.metrics import r2_score,f1_score,accuracy_score,roc_auc_score,recall_score,mean_squared_error
print('r2_score',r2_score(df_wf_y_oh.ravel(),pre.ravel()))
print('roc_auc_score',roc_auc_score(df_wf_y_oh.ravel(),pre.ravel()))
print('mean_squared_error',mean_squared_error(df_wf_y_oh.ravel(),pre.ravel()))

# correct_pred = np.equal(np.argmax(y, 1), np.argmax(pre, 1))
pre_max = np.argmax(pre, 1)
print(np.argmax(pre, 1))

# Classification
print('recall_score',recall_score(df_wf_y.ravel(),pre_max.ravel()))
print('f1_score',f1_score(df_wf_y.ravel(),pre_max.ravel()))
print('accuracy_score',accuracy_score(df_wf_y.ravel(),pre_max.ravel()))

In [ ]:
#####无视下面########

In [314]:
np_x_train, np_x_test, np_y_train, np_y_test = train_test_split(
    df_house_x, df_house_y, test_size=0.7)

np_y_train = np.array(np_y_train)
np_y_test = np.array(np_y_test)
# np_y_train = np_y_train.values.reshape(-1,1)
# np_y_test = np_y_test.values.reshape(-1,1)

In [315]:
ss = StandardScaler()
np_x_train = ss.fit_transform(np_x_train)
np_x_test = ss.fit_transform(np_x_test)

In [331]:
#回归，非独热试试
#结论：非连续数值，不适合回归。会固定再一个值附近徘徊。
input_size = 3
output_size = 1
batch_size = 1000
# placeholder
x = tf.placeholder(tf.float32, [None, input_size], name='x')
y = tf.placeholder(tf.float32, [None], name='y')

# Variable
W1 = tf.Variable(
    tf.random_normal([input_size, input_size], stddev=0.03), name='W1')
b1 = tf.Variable(tf.zeros([input_size]), name='b1')
h1 = tf.nn.relu(tf.add(tf.matmul(x, W1), b1))

W2 = tf.Variable(
    tf.random_normal([input_size, output_size], stddev=0.03), name='W2')
b2 = tf.Variable(tf.zeros([output_size]), name='b2')
# ho = tf.add(tf.matmul(h1, W2), b2)

prediction = tf.add(tf.matmul(h1, W2), b2)
# cross_entropy = tf.reduce_mean(
#     tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=ho))
cross_entropy = tf.reduce_mean(tf.reduce_sum(tf.square(y-prediction), reduction_indices=[1]))
optimizer = tf.train.AdamOptimizer(0.003).minimize(cross_entropy)

correct_pred = tf.equal(y, prediction)
acc_op = tf.reduce_mean(tf.cast(correct_pred, tf.float32))



In [337]:
# tf.reset_default_graph()
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    
    total_batch_len = int(len(np_x_train) / batch_size)
    for i in range(100):
        for j  in range(total_batch_len):
            batch_x = np_x_train[j*batch_size : (j+1)*batch_size]
            batch_y = np_y_train[j*batch_size : (j+1)*batch_size]
            _,loss = sess.run([optimizer,cross_entropy],feed_dict={x:batch_x,y:batch_y})
#         _,loss = sess.run([optimizer,cross_entropy],feed_dict={x:np_x_train,y:np_y_train})
        if(i % 10 == 0):
            print("i:", i, "loss = ", "{:.3f}".format(loss))
    
#     acc = sess.run(accuracy, feed_dict={x: np_x_test, y: np_y_test})
#     print(acc)
    pre = sess.run(prediction,feed_dict={x:np_x_test})
    print(pre)
    
#     accuracy = sess.run(acc_op,feed_dict={x:np_x_test,y:np_y_test})
#     print("Accuracy on validation set: %.9f" % accuracy)

i: 0 loss =  442.081
i: 10 loss =  278.407
i: 20 loss =  251.895
i: 30 loss =  248.487
i: 40 loss =  248.480
i: 50 loss =  248.480
i: 60 loss =  248.480
i: 70 loss =  248.480
i: 80 loss =  248.480
i: 90 loss =  248.480
[[0.46168822]
 [0.4617226 ]
 [0.4617075 ]
 ...
 [0.46158308]
 [0.46156523]
 [0.46159333]]


In [342]:
# np.savetxt("np_host.txt",pre)
print(np.max(pre))
print(np.min(pre))

0.46272212
0.46138948


In [3]:



import sys

print(sys.path)

['', '/anaconda3/lib/python36.zip', '/anaconda3/lib/python3.6', '/anaconda3/lib/python3.6/lib-dynload', '/anaconda3/lib/python3.6/site-packages', '/anaconda3/lib/python3.6/site-packages/aeosa', '/anaconda3/lib/python3.6/site-packages/IPython/extensions', '/Users/hzy/.ipython']


In [1]:
# Creates a graph.
with tf.device('/cpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

NameError: name 'tf' is not defined